In [1]:
import numpy as np

from importdata import data
from fft import *
from mobility import *
from permutationentropy import *
from utils import *
from lempelziv import *
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from parameters import *


The BIS data is universally longer than the corresponding EEG data. 
It is assumed that the recording of the BIS data started first, and
this cell correspondingly trims the BIS data.

In [2]:
for case in data:
    eeg_duration = case['EEG'].shape[1]/sampling_rate
    print(eeg_duration)
    bis_duration = case['bis'].shape[1]/bis_sampling_rate
    print(bis_duration)
    difference = bis_duration - eeg_duration
    trim_length = int(round(difference * bis_sampling_rate))
    case['bis'] = case['bis'][:, trim_length:]
    bis_duration = case['bis'].shape[1]/bis_sampling_rate
    print(bis_duration)



3760.8125
3835.0
3760.0
4888.0
5010.0
4890.0
13071.5
13380.0
13070.0
3898.75
4010.0
3900.0
2419.75
2490.0
2420.0
5026.5
5175.0
5025.0
6054.75
6205.0
6055.0
7949.75
8145.0
7950.0
5977.90625
6125.0
5980.0
6199.75
6365.0
6200.0
13414.0
13750.0
13415.0
20338.25
20865.0
20340.0
4090.15625
4170.0
4090.0
3117.4140625
3205.0
3115.0
13180.25
13505.0
13180.0
7170.0
7340.0
7170.0
7598.75
7785.0
7600.0
3705.75
3800.0
3705.0
4355.0
4530.0
4355.0
9749.65625
9985.0
9750.0
9357.5
9590.0
9360.0
2314.0
2380.0
2315.0
5821.75
5980.0
5820.0
3778.5
3875.0
3780.0


In [3]:
train_data = data[::2] #select every second case for training
test_data = data[1::2] #select every other case for testing

    
eeg_train_data = [case['EEG'] for case in train_data]
bis_train_data = [case['bis'] for case in train_data]

eeg_test_data = [case['EEG'] for case in test_data]
bis_test_data = [case['bis'] for case in train_data]

signal_lengths = [len(case[0]) for case in eeg_train_data] 


In [4]:
decomposed_eeg_train_data = [fft(case) for case in eeg_train_data] #returns an array where each element represents a case and each case is a dictionary where the keys are frequency bands and the values are in the frequency domain
decomposed_eeg_train_time_domain = convert_decomposed_to_time_domain(decomposed_eeg_train_data) #the same as above except the values are in the time domain
decomposed_eeg_train_time_domain_power = convert_decomposed_amplitude_to_power(decomposed_eeg_train_time_domain) #the same as above except the values represent power rather than amplitude

decomposed_eeg_test_data = [fft(case) for case in eeg_test_data]
decomposed_eeg_test_time_domain = convert_decomposed_to_time_domain(decomposed_eeg_test_data)
decomposed_eeg_test_time_domain_power = convert_decomposed_amplitude_to_power(decomposed_eeg_test_time_domain)

In [5]:
mobility_amplitude_array_train = operation_on_multiple_case_data(decomposed_eeg_train_time_domain, calculate_mobility, sampling_rate, signal_lengths)
pe_amplitude_array_train = operation_on_multiple_case_data(decomposed_eeg_train_time_domain, PE, sampling_rate, signal_lengths)
lzc_amplitude_array_train = operation_on_multiple_case_data(decomposed_eeg_train_time_domain, LZC_single_band, sampling_rate, signal_lengths)

mobility_power_array_train = operation_on_multiple_case_data(decomposed_eeg_train_time_domain_power, calculate_mobility, sampling_rate, signal_lengths)
pe_power_array_train = operation_on_multiple_case_data(decomposed_eeg_train_time_domain_power, PE, sampling_rate, signal_lengths)
lzc_power_array_train = operation_on_multiple_case_data(decomposed_eeg_train_time_domain_power, LZC_single_band, sampling_rate, signal_lengths)

In [ ]:
mobility_amplitude_bis_train_dict = associate_features_with_BIS(mobility_amplitude_array_train, bis_train_data, mobility_window_duration, mobility_overlap_fraction, bis_sampling_rate)
mobility_amplitude_train_r2 = linear_regression(mobility_amplitude_bis_train_dict)
print(mobility_amplitude_train_r2)

Delta
Coefficient: [-15.42194059]
Intercept: 54.610204240141776
Theta
Coefficient: [-12.38183902]
Intercept: 52.3914559479907
Alpha
Coefficient: [-7.99149435]
Intercept: 49.28325146979309
Beta
Coefficient: [36.89639016]
Intercept: 19.30691889618328
Gamma
Coefficient: [-130.54291046]
Intercept: 135.98767190092508
{'Delta': 0.0021115154812348935, 'Theta': 0.000717267366801777, 'Alpha': 0.00019929640791160885, 'Beta': 0.007488175856813717, 'Gamma': 0.050570802075941224}


In [8]:
mobility_power_bis_train_dict = associate_features_with_BIS(mobility_power_array_train, bis_train_data, mobility_window_duration, mobility_overlap_fraction, bis_sampling_rate)
mobility_power_train_r2 = linear_regression(mobility_power_bis_train_dict)
print(mobility_power_train_r2)

frequency_band: Delta
length of unscaled feature data: 5558
length of untrimmed bis: 752
length of unscaled feature data: 13016
length of untrimmed bis: 2614
length of unscaled feature data: 2365
length of untrimmed bis: 484
length of unscaled feature data: 8999
length of untrimmed bis: 1211
length of unscaled feature data: 5923
length of untrimmed bis: 1196
length of unscaled feature data: 13359
length of untrimmed bis: 2683
length of unscaled feature data: 6052
length of untrimmed bis: 818
length of unscaled feature data: 19687
length of untrimmed bis: 2636
length of unscaled feature data: 11315
length of untrimmed bis: 1520
length of unscaled feature data: 4300
length of untrimmed bis: 871
length of unscaled feature data: 9302
length of untrimmed bis: 1872
length of unscaled feature data: 5767
length of untrimmed bis: 1164
frequency_band: Theta
length of unscaled feature data: 4323
length of untrimmed bis: 752
length of unscaled feature data: 15186
length of untrimmed bis: 2614
leng

In [9]:
pe_amplitude_bis_train_dict = associate_features_with_BIS(pe_amplitude_array_train, bis_train_data, complexity_features_window, complexity_features_overlap, bis_sampling_rate)
pe_amplitude_train_r2 = linear_regression(pe_amplitude_bis_train_dict)
print(pe_amplitude_train_r2)

frequency_band: Delta
length of unscaled feature data: 1610
length of untrimmed bis: 752
length of unscaled feature data: 5601
length of untrimmed bis: 2614
length of unscaled feature data: 1036
length of untrimmed bis: 484
length of unscaled feature data: 2593
length of untrimmed bis: 1211
length of unscaled feature data: 2560
length of untrimmed bis: 1196
length of unscaled feature data: 5747
length of untrimmed bis: 2683
length of unscaled feature data: 1751
length of untrimmed bis: 818
length of unscaled feature data: 5647
length of untrimmed bis: 2636
length of unscaled feature data: 3255
length of untrimmed bis: 1520
length of unscaled feature data: 1865
length of untrimmed bis: 871
length of unscaled feature data: 4009
length of untrimmed bis: 1872
length of unscaled feature data: 2494
length of untrimmed bis: 1164
frequency_band: Theta
length of unscaled feature data: 1644
length of untrimmed bis: 752
length of unscaled feature data: 5717
length of untrimmed bis: 2614
length of

ValueError: Found input variables with inconsistent numbers of samples: [17819, 19340]